In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
url_list = []

for i in list(range(1, 87)):
    url = "https://www.dermstore.com/skin-care.list?pageNumber=" + str(i)
    url_list.append(url)    

In [3]:
results_list = []
for url in url_list:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all("li", class_ = "productListProducts_product")
    results_list.append(results)

In [4]:
results_string = str(results_list)
results_string = results_string.split("productBlock_productName")

In [13]:
product_list = []
for ele in results_string:
    ele = ele.split("data")
    name = list(set([string for string in ele if "-product-name" in string]))
    brand = list(set([string for string in ele if "-product-brand" in string]))
    price = list(set([string for string in ele if "-product-price" and "$" in string]))
    url = list(set([string for string in ele if "href" and ".html" in string]))
    dict_i = {"product_name": name, "brand": brand, "price": price, "url": url}
    product_list.append(dict_i)

In [14]:
products_df = pd.DataFrame(product_list)
products_df = products_df[products_df["product_name"].map(lambda d: len(d)) > 0].reset_index()
products_df = products_df.applymap(lambda x: x[0] if isinstance(x, list) else x)

products_df["product_name"] = products_df["product_name"].str.replace("-product-name=", "")
products_df["product_name"] = products_df["product_name"].str.replace('"', "")

products_df["brand"] = products_df["brand"].str.replace("-product-brand=", "")
products_df["brand"] = products_df["brand"].str.replace('"', "")

products_df["price"] = products_df["price"].str.replace("-product-price=", "")
products_df["price"] = products_df["price"].str.replace('"', "")

products_df["url"] = products_df["url"].apply(lambda x: x.split("href")[-1])
products_df["url"] = products_df["url"].str.replace("=", "")
products_df["url"] = products_df["url"].str.replace('"', "")
products_df["url"] = products_df["url"].str.replace(">\n<div classproductBlock_title>\n<h3 class", "")
products_df["url"] = products_df["url"].str.replace(">\n<div classproductBlock_imageContainer ", "")
products_df["url"] = "https://www.dermstore.com" + products_df["url"]

products_df = products_df.drop(columns = "index")

In [15]:
products_df

,product_name,brand,price,url
0,ELEMIS Pro-Collagen Cleansing Balm (100 g.),Elemis,$68.00,https://www.dermstore.com/elemis-pro-collagen-...
1,VIRTUE Healing Oil (1.7 fl. oz.),VIRTUE,$44.00,https://www.dermstore.com/virtue-healing-oil-5...
2,VIRTUE Flourish Nightly Intensive Hair Growth ...,VIRTUE,$134.00,https://www.dermstore.com/virtue-flourish-nigh...
3,Augustinus Bader The Rich Cream 50ml,Augustinus Bader,$290.00,https://www.dermstore.com/augustinus-bader-the...
4,SkinCeuticals C E Ferulic (1 fl. oz.),SkinCeuticals,$182.00,https://www.dermstore.com/skinceuticals-c-e-fe...
...,...,...,...,...
4330,LightStim LightStim for Acne (1 kit),LightStim,$169.00,https://www.dermstore.com/lightstim-for-acne-l...
4331,LightStim LightStim PhotoSerum (1 fl. oz.),LightStim,$80.00,https://www.dermstore.com/lightstim-photoserum...
4332,FOREO BEAR mini 1 piece,FOREO,$219.00,https://www.dermstore.com/foreo-bear-mini-faci...
4333,Murad Hydrate Trial Kit 2.83 fl. oz. - $58 Value,Murad,$36.00,https://www.dermstore.com/murad-hydrate-trial-...


In [16]:
products_df.to_csv(r"data/dermstore_basic.csv")

In [17]:
url_list2 = products_df["url"].to_list()

In [22]:
results_list = []
for url in url_list2[:1000]:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all("div", class_ = "athenaProductPage_breakpoint-lg_productDescription")
    results_list.append(results)

KeyboardInterrupt: 

In [29]:
results_list[0]

[<div class="athenaProductPage_breakpoint-lg_productDescription">
 <div class="athenaProductPage_productBuyNowPayLaterProviders">
 <ul aria-label="buy now pay later options" class="productBuyNowPayLater desktopView" data-component="productBuyNowPayLaterProviders" data-isdesktop="true" data-productprice="68.0">
 <li aria-label="klarna_slice" class="productBuyNowPayLater_provider">
 <div class="productBuyNowPayLater_provider_logo"><svg height="20" role="img" viewbox="0 0 50 20" width="50" xmlns="http://www.w3.org/2000/svg">
 <title>Klarna Slice</title>
 <rect fill="#ffb3c7" height="20" rx="4" width="50"></rect>
 <path d="M 39.871094 8.761719 C 38.59375 7.894531 36.871094 8.109375 35.851562 9.269531 C 34.828125 10.425781 34.828125 12.164062 35.851562 13.320312 C 36.871094 14.476562 38.59375 14.695312 39.871094 13.824219 L 39.871094 14.195312 L 41.609375 14.195312 L 41.609375 8.390625 L 39.871094 8.390625 Z M 38.292969 12.789062 C 37.476562 12.746094 36.847656 12.054688 36.875 11.242188 C 

In [30]:
results_list = [str(ele) for ele in results_list]

In [44]:
results_list[0].split("athenaProductPageSynopsisContent")[4].split("productDescription_contentPropertyValue_value")[-1]

'">105g</li>\n</ul>\n</div>\n</div>\n</div>\n</div>\n</div>\n</div>\n</div>]'

In [68]:
product_list1 = []
description = [string for string in results_list[0].split("div") if "athenaProductPageSynopsisContent" in string]
dict_i = {"description": description}
product_list1.append(dict_i)

In [69]:
products_df1 = pd.DataFrame(product_list1)

In [70]:
products_df1 = products_df1.explode("description")

In [71]:
products_df1["description"].iloc[1]

' class="athenaProductPageSynopsisContent"><p>Use daily as a cleanser or weekly as a nourishing facial mask. Step 1: Warm a pea-sized amount between palms and then massage over face, neck and décolleté with light circular massage movements. Step 2: Moisten fingertips and continue to massage. Remove with a warm, damp cleansing cloth.</p>\n</'

In [49]:
product_list1 = []
for product in results_list:
    product = [ele.split("div") for ele in product]
    description = product[1]
    ingredients = product[-1]
    dict_i = {"description": description, "ingredients": ingredients}
    product_list1.append(dict_i)

In [51]:
products_df1 = pd.DataFrame(product_list1)